In [ ]:
import numba as nb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Times New Roman']
plt.rcParams['figure.dpi'] = 300
import seaborn as sns
from scipy.interpolate import griddata
import os
import shutil
import moviepy as mp

In [ ]:
SIM_NAME = 'Sim 2'

data = pd.read_csv(f'Output/{SIM_NAME}.csv', header=0)
data

In [ ]:
data['pos_rms'] = data['pos_rms'].fillna(1)
data['mag_rms'] = data['mag_rms'].fillna(6000)